# Data Preparation

In [19]:
import re

import numpy  as np
import pandas as pd

In [20]:
def clean_data_fn(text):
    """Clean and convert a text to store only alphabetical characters 
       in lower case.
       
    Args:
        text (str): a text string.
    
    Returns:
        text (str): a text string converted.
    """
    text = text.lower()
    text = re.sub(r"([?.!,¿])", r" ", text)
    text = re.sub(r'[" "]+', " ", text)
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text=re.sub(r'@\w+', '',text)
    return text

def load_data_fn(file_path):
    """Read a bug report data set.

    Args:
        filepath (str): a complete filename path.

    Returns:
        result (dataframe): a bug report dataframe.

    """
    reports = pd.read_csv(file_path, encoding='utf8', sep=',', parse_dates=True
      ,low_memory=False)

    reports.dropna(inplace=True)
    reports['long_description'] = reports['long_description'].map(clean_data_fn)
    reports['long_description'] = reports['long_description'].replace('', np.nan)
    
    result = reports.loc[:, ('long_description', 'severity_category')]
    result.dropna(inplace=True)
    result.reset_index(drop=True, inplace=True)

    return result

def convert_to_ordinal_fn(severity):
    """Convert severity category to ordinal.

    Args:
        severity (str): a severity category.

    Returns:
        severity (int): ordinal value of severity.
    """
    categories={'trivial': 0,
                'minor': 1,
                'major': 2,
                'critical': 3,
                'blocker': 4}
    return categories.get(severity, 'Invalid severity category!')

    

In [22]:
reports_input_url = 'https://raw.githubusercontent.com/gomesluiz/bug-severity-predictor/main/data/raw/mozilla_bug_report_data.csv?token=AAGZN3HE4XJHL3C333SMR53ABL5VI'
!mkdir -p content/data/raw
!wget -O content/data/raw/mozilla_bug_report_data.csv {reports_input_url}
#!head -2 {reports_input_path}

--2021-01-22 16:20:07--  https://raw.githubusercontent.com/gomesluiz/bug-severity-predictor/main/data/raw/mozilla_bug_report_data.csv?token=AAGZN3HE4XJHL3C333SMR53ABL5VI
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2077348 (2.0M) [text/plain]
Saving to: ‘content/data/raw/mozilla_bug_report_data.csv’

content/data/raw/mo 100%[===================>]   1.98M  10.9MB/s    in 0.2s    

2021-01-22 16:20:08 (10.9 MB/s) - ‘content/data/raw/mozilla_bug_report_data.csv’ saved [2077348/2077348]



In [23]:
reports_input_path =  'content/data/raw/mozilla_bug_report_data.csv'
reports_data = load_data_fn(reports_input_path)

In [24]:
reports_data.head()

,long_description,severity_category
0,is broken many users can t enter bugs on it p...,blocker
1,adding support for custom headers and cookie n...,blocker
2,the patch in bug regressed the fix from bug th...,major
3,from bugzilla helper user agent mozilla x u li...,major
4,i found it odd that relogin cgi didn t clear o...,minor


In [25]:
reports_data['severity_category'].value_counts()

major       737
critical    605
minor       540
trivial     302
blocker     204
Name: severity_category, dtype: int64

In [26]:
reports_data['severity_code'] = reports_data['severity_category'].apply(convert_to_ordinal_fn) 

In [27]:
reports_data.head()

,long_description,severity_category,severity_code
0,is broken many users can t enter bugs on it p...,blocker,4
1,adding support for custom headers and cookie n...,blocker,4
2,the patch in bug regressed the fix from bug th...,major,2
3,from bugzilla helper user agent mozilla x u li...,major,2
4,i found it odd that relogin cgi didn t clear o...,minor,1


In [28]:
reports_data['severity_code'].value_counts()

2    737
3    605
1    540
0    302
4    204
Name: severity_code, dtype: int64

In [29]:
!mkdir -p content/data/clean
reports_output_path = 'content/data/clean/mozilla_bug_report_data.csv'
reports_data[['long_description', 'severity_code']].to_csv(reports_output_path, index=False)

In [30]:
!head -2 {reports_output_path}

long_description,severity_code
 is broken many users can t enter bugs on it particularly not from a fresh install so we need to pull and post a instead ,4
